In [29]:
# Step 1: Install required packages
!pip install fastapi uvicorn streamlit pyngrok plotly streamlit-lottie -q

# Step 2: Import dependencies
import os
import time
import subprocess
import requests
from pyngrok import ngrok

# Step 3: Kill any existing processes
print("Killing existing processes...")
subprocess.run(["pkill", "-f", "uvicorn"], check=False)
subprocess.run(["pkill", "-f", "streamlit"], check=False)
subprocess.run(["pkill", "-f", "ngrok"], check=False)
time.sleep(3)

# Step 4: Create the main.py file with the backend code
main_py_code = """
import numpy as np
import pandas as pd
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Dict, Optional
from fastapi.middleware.cors import CORSMiddleware
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Create FastAPI app
app = FastAPI(title="AI Virtual Stylist", description="Personalized outfit recommendation system")

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Create enhanced mock dataset
def create_mock_data():
    body_shapes = {
        'apple': {'description': 'Wider torso, narrower hips', 'tips': 'Draw attention away from midsection'},
        'pear': {'description': 'Narrower torso, wider hips', 'tips': 'Balance proportions with structured tops'},
        'hourglass': {'description': 'Balanced bust and hips with narrow waist', 'tips': 'Highlight waist with fitted styles'},
        'rectangle': {'description': 'Similar width bust, waist, and hips', 'tips': 'Create curves with strategic layering'},
        'inverted_triangle': {'description': 'Wider shoulders, narrower hips', 'tips': 'Add volume to lower body'}
    }
    styles = {
        'casual': {'description': 'Comfortable, relaxed, everyday wear', 'colors': 'earth tones, pastels, denim'},
        'formal': {'description': 'Professional, elegant attire', 'colors': 'navy, black, gray, white'},
        'bohemian': {'description': 'Free-spirited, artistic, unconventional', 'colors': 'warm tones, patterns, textured fabrics'},
        'sporty': {'description': 'Athletic, functional, active wear', 'colors': 'brights, neutrals, performance fabrics'},
        'classic': {'description': 'Timeless, tailored, traditional', 'colors': 'neutrals, jewel tones, clean lines'},
        'trendy': {'description': 'Fashion-forward, current styles', 'colors': 'seasonal colors, bold patterns, statement pieces'}
    }
    events = {
        'work': {'dress_code': 'Business casual to formal', 'formality': 'medium'},
        'party': {'dress_code': 'Cocktail to festive', 'formality': 'high'},
        'date': {'dress_code': 'Smart casual to dressy', 'formality': 'medium'},
        'casual outing': {'dress_code': 'Relaxed and comfortable', 'formality': 'low'},
        'wedding': {'dress_code': 'Formal to black tie', 'formality': 'very high'},
        'interview': {'dress_code': 'Professional and conservative', 'formality': 'high'}
    }
    categories = ['top', 'bottom', 'dress', 'outerwear', 'shoes', 'accessories']
    colors = {
        'black': '#000000', 'white': '#FFFFFF', 'blue': '#1E88E5', 'red': '#E53935',
        'green': '#43A047', 'yellow': '#FDD835', 'purple': '#8E24AA', 'pink': '#D81B60',
        'brown': '#6D4C41', 'gray': '#757575'
    }
    items = []
    for i in range(1000):
        category = np.random.choice(categories)
        color_name = np.random.choice(list(colors.keys()))
        style = np.random.choice(list(styles.keys()))
        if category == 'top':
            name_options = ['Silk Blouse', 'Cotton Tee', 'Cashmere Sweater', 'Button-Down Shirt', 'Crop Top']
        elif category == 'bottom':
            name_options = ['Skinny Jeans', 'Tailored Trousers', 'A-Line Skirt', 'Pleated Pants', 'Culottes']
        elif category == 'dress':
            name_options = ['Wrap Dress', 'Shift Dress', 'Maxi Dress', 'Bodycon Dress', 'Shirt Dress']
        elif category == 'outerwear':
            name_options = ['Trench Coat', 'Leather Jacket', 'Blazer', 'Wool Coat', 'Denim Jacket']
        elif category == 'shoes':
            name_options = ['Ankle Boots', 'Pumps', 'Sneakers', 'Sandals', 'Loafers']
        else:
            name_options = ['Statement Necklace', 'Silk Scarf', 'Leather Belt', 'Tote Bag', 'Sunglasses']
        item = {
            'id': i,
            'name': f"{np.random.choice(name_options)} #{i}",
            'category': category,
            'color': color_name,
            'color_hex': colors[color_name],
            'style': style,
            'suitable_body_shapes': np.random.choice(list(body_shapes.keys()), size=np.random.randint(1, 4)),
            'suitable_events': np.random.choice(list(events.keys()), size=np.random.randint(1, 4)),
            'trend_score': np.random.uniform(0, 1),
            'seasonality': np.random.choice(['all', 'spring', 'summer', 'fall', 'winter']),
            'description': f"Perfect {category} for {np.random.choice(list(events.keys()))}",
            'brand': np.random.choice(['Zara', 'H&M', 'Gucci', 'Nike', 'Adidas', 'Levi\\'s', 'Uniqlo', 'Mango', 'Forever 21']),
            'price': np.random.randint(20, 300),
            'material': np.random.choice(['Cotton', 'Silk', 'Polyester', 'Wool', 'Denim', 'Leather']),
            'care_instructions': np.random.choice(['Machine wash', 'Dry clean', 'Hand wash', 'Spot clean'])
        }
        items.append(item)
    return pd.DataFrame(items), body_shapes, styles, events, colors

# Initialize data
try:
    clothing_data, body_shapes, styles, events, colors = create_mock_data()
    logger.info("Mock data created successfully")
except Exception as e:
    logger.error(f"Error creating mock data: {str(e)}")

# Define API models
class StyleRequest(BaseModel):
    body_shape: str
    personal_style: str
    event_type: str
    exclude_colors: Optional[List[str]] = []
    budget: Optional[int] = None

class OutfitItem(BaseModel):
    id: int
    name: str
    category: str
    color: str
    color_hex: str
    style: str
    reason: str
    brand: str
    price: int
    description: str
    material: str
    care_instructions: str
    compatibility_score: float

class OutfitRecommendation(BaseModel):
    outfit: List[OutfitItem]
    overall_compatibility: float
    message: str
    body_shape_info: Dict[str, str]
    style_info: Dict[str, str]
    event_info: Dict[str, str]
    total_price: int

# Recommendation function with simplified scoring
def recommend_outfits(request: StyleRequest):
    try:
        logger.info(f"Processing request: {request}")

        if request.body_shape not in body_shapes:
            raise HTTPException(status_code=400, detail=f"Invalid body shape. Valid options: {list(body_shapes.keys())}")
        if request.personal_style not in styles:
            raise HTTPException(status_code=400, detail=f"Invalid personal style. Valid options: {list(styles.keys())}")
        if request.event_type not in events:
            raise HTTPException(status_code=400, detail=f"Invalid event type. Valid options: {list(events.keys())}")

        # Filter data based on request
        filtered_data = clothing_data[
            clothing_data['suitable_body_shapes'].apply(lambda x: request.body_shape in x) &
            clothing_data['suitable_events'].apply(lambda x: request.event_type in x)
        ]

        if request.exclude_colors:
            filtered_data = filtered_data[~filtered_data['color'].isin(request.exclude_colors)]
        if request.budget:
            filtered_data = filtered_data[filtered_data['price'] <= request.budget]

        if filtered_data.empty:
            raise HTTPException(status_code=404, detail="No items found matching your criteria. Try adjusting your filters.")

        # Calculate compatibility score (simplified without TensorFlow)
        filtered_data['score'] = 0.0
        for idx, item in filtered_data.iterrows():
            # Base score for matching body shape and event
            score = 0.7

            # Bonus for matching style
            if item['style'] == request.personal_style:
                score += 0.2

            # Add trendiness component
            score += item['trend_score'] * 0.1

            filtered_data.at[idx, 'score'] = min(score, 1.0)  # Cap at 1.0

        filtered_data = filtered_data.sort_values(by='score', ascending=False)

        # Create outfit recommendations
        outfit_items, total_price = [], 0
        for category in ['top', 'bottom', 'dress', 'outerwear', 'shoes', 'accessories']:
            category_items = filtered_data[filtered_data['category'] == category]
            if not category_items.empty:
                top_item = category_items.iloc[0]
                reason = f"Perfect for your {request.body_shape} body shape. Matches your {request.personal_style} style and the {request.event_type} event. Trendiness score: {top_item['trend_score']:.2f}"
                outfit_items.append(OutfitItem(
                    id=int(top_item['id']),
                    name=top_item['name'],
                    category=top_item['category'],
                    color=top_item['color'],
                    color_hex=top_item['color_hex'],
                    style=top_item['style'],
                    reason=reason,
                    brand=top_item['brand'],
                    price=int(top_item['price']),
                    description=top_item['description'],
                    material=top_item['material'],
                    care_instructions=top_item['care_instructions'],
                    compatibility_score=float(top_item['score'])
                ))
                total_price += top_item['price']

        overall_score = filtered_data['score'].mean()
        if overall_score > 0.8:
            message = "Excellent match! This outfit is perfect for you and will make you look amazing."
        elif overall_score > 0.6:
            message = "Great choice! This outfit suits you well and matches your preferences."
        else:
            message = "Good option! You might want to consider some accessories to complete the look."

        logger.info("Recommendation generated successfully")
        return OutfitRecommendation(
            outfit=outfit_items,
            overall_compatibility=float(overall_score),
            message=message,
            body_shape_info=body_shapes[request.body_shape],
            style_info=styles[request.personal_style],
            event_info=events[request.event_type],
            total_price=int(total_price)
        )
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"Error in recommendation: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Internal Server Error: {str(e)}")

# API endpoints
@app.post("/recommend", response_model=OutfitRecommendation)
def get_recommendation(request: StyleRequest):
    return recommend_outfits(request)

@app.get("/health")
def health_check():
    return {"status": "healthy"}

@app.get("/options")
def get_options():
    return {
        "body_shapes": list(body_shapes.keys()),
        "styles": list(styles.keys()),
        "events": list(events.keys()),
        "colors": list(colors.keys())
    }

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
"""

with open("main.py", "w") as f:
    f.write(main_py_code)

# Step 5: Create the app.py file with the frontend code
app_py_code = """
import streamlit as st
import requests
import json
import os
import plotly.graph_objects as go
import plotly.express as px
from streamlit_lottie import st_lottie

# Set page configuration
st.set_page_config(
    page_title="AI Virtual Stylist",
    page_icon="👗",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Load Lottie animation safely with requests
def load_lottieurl(url: str):
    try:
        r = requests.get(url)
        if r.status_code == 200:
            return r.json()
    except Exception as e:
        st.error(f"Error loading Lottie: {e}")
    return None

# Load animations
lottie_robot = load_lottieurl("https://assets1.lottiefiles.com/packages/lf20_touohxv0.json")
lottie_fashion = load_lottieurl("https://assets9.lottiefiles.com/packages/lf20_V9t630.json")

# API URL (will be replaced by ngrok URL)
api_url = os.getenv("BACKEND_URL", "http://localhost:8000/recommend")

# Custom CSS
st.markdown(r\"\"\"
<style>
    @import url('https://fonts.googleapis.com/css2?family=Montserrat:wght@300;400;500;600;700&display=swap');
    html, body, [class*="css"] { font-family: 'Montserrat', sans-serif; }

    /* Color indicator style */
    .color-indicator {
        display: inline-block;
        width: 20px;
        height: 20px;
        border-radius: 50%;
        margin-right: 10px;
        vertical-align: middle;
    }

    /* Card style */
    .item-card {
        padding: 15px;
        border-radius: 10px;
        background-color: #f8f9fa;
        margin-bottom: 15px;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }
</style>
\"\"\", unsafe_allow_html=True)

st.title("👗 AI Virtual Stylist")
st.write("Personalized outfit recommendations based on your body shape, style, and event.")

# Sidebar input form
st.sidebar.header("Customize Your Outfit")
body_shape = st.sidebar.selectbox("Choose Body Shape", ["apple", "pear", "hourglass", "rectangle", "inverted_triangle"])
personal_style = st.sidebar.selectbox("Choose Personal Style", ["casual", "formal", "bohemian", "sporty", "classic", "trendy"])
event_type = st.sidebar.selectbox("Choose Event Type", ["work", "party", "date", "casual outing", "wedding", "interview"])
budget = st.sidebar.slider("Budget ($)", 20, 500, 150)
exclude_colors = st.sidebar.multiselect("Exclude Colors", ["black", "white", "blue", "red", "green", "yellow", "purple", "pink", "brown", "gray"])

# Button to get recommendations
if st.sidebar.button("Get Recommendations"):
    with st.spinner("Finding your perfect outfit..."):
        payload = {
            "body_shape": body_shape,
            "personal_style": personal_style,
            "event_type": event_type,
            "budget": budget,
            "exclude_colors": exclude_colors
        }
        try:
            response = requests.post(api_url, json=payload, timeout=30)
            if response.status_code == 200:
                rec = response.json()
                st.subheader("✨ Outfit Recommendation")
                st.markdown(f"**Overall Compatibility:** {rec['overall_compatibility']:.2f}")
                st.markdown(f"**Message:** {rec['message']}")

                # Display outfit items in a clean card layout
                for item in rec["outfit"]:
                    st.markdown(f\"\"\"
                    <div class="item-card">
                        <h3>{item['name']}</h3>
                        <p><strong>Category:</strong> {item['category']}</p>
                        <p><strong>Color:</strong> <span class="color-indicator" style="background-color: {item['color_hex']}"></span>{item['color']}</p>
                        <p><strong>Brand:</strong> {item['brand']}</p>
                        <p><strong>Price:</strong> ${item['price']}</p>
                        <p><strong>Material:</strong> {item['material']}</p>
                        <p><strong>Care:</strong> {item['care_instructions']}</p>
                        <p><strong>Compatibility Score:</strong> {item['compatibility_score']:.2f}</p>
                        <p><strong>Why it works:</strong> {item['reason']}</p>
                    </div>
                    \"\"\", unsafe_allow_html=True)

                # Display total price
                st.markdown(f"**Total Price:** ${rec['total_price']}")

                # Display additional information
                with st.expander("Style & Event Information"):
                    st.markdown(f"**Body Shape Info:**")
                    st.markdown(f"- {rec['body_shape_info']['description']}")
                    st.markdown(f"- Tip: {rec['body_shape_info']['tips']}")

                    st.markdown(f"**Style Info:**")
                    st.markdown(f"- {rec['style_info']['description']}")
                    st.markdown(f"- Colors: {rec['style_info']['colors']}")

                    st.markdown(f"**Event Info:**")
                    st.markdown(f"- Dress code: {rec['event_info']['dress_code']}")
                    st.markdown(f"- Formality: {rec['event_info']['formality']}")
            else:
                st.error(f"Error fetching recommendations. Status: {response.status_code}. Response: {response.text}")
        except requests.exceptions.RequestException as e:
            st.error(f"API request failed: {e}")
"""

with open("app.py", "w") as f:
    f.write(app_py_code)

# Step 6: Set ngrok auth token
ngrok.set_auth_token("32Dvog6ji9AkRGzWZ8CQp2EYS7U_2kbtWYbhpNPpL637Z4LE7")

# Step 7: Start FastAPI backend
print("Starting FastAPI backend...")
backend_process = subprocess.Popen(["python", "main.py"])
time.sleep(5)  # Give the backend time to start

# Step 8: Start ngrok tunnel for FastAPI
print("Starting ngrok tunnel for FastAPI...")
fastapi_tunnel = ngrok.connect(8000)
print("FastAPI backend URL:", fastapi_tunnel.public_url)

# Step 9: Set environment variable for backend URL
os.environ["BACKEND_URL"] = f"{fastapi_tunnel.public_url}/recommend"

# Step 10: Start Streamlit frontend
print("Starting Streamlit frontend...")
frontend_process = subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501"])
time.sleep(5)  # Give the frontend time to start

# Step 11: Start ngrok tunnel for Streamlit
print("Starting ngrok tunnel for Streamlit...")
streamlit_tunnel = ngrok.connect(8501)
print("Streamlit frontend URL:", streamlit_tunnel.public_url)

print("\nServices are running...")
print("Streamlit app:", streamlit_tunnel.public_url)
print("FastAPI docs:", fastapi_tunnel.public_url + "/docs")
print("Backend health check:", fastapi_tunnel.public_url + "/health")

# Step 12: Test the backend health
try:
    response = requests.get(f"{fastapi_tunnel.public_url}/health")
    print(f"\nHealth check status: {response.status_code}")
    print(f"Response: {response.text}")

    # Test with sample data
    payload = {
        "body_shape": "hourglass",
        "personal_style": "classic",
        "event_type": "work",
        "budget": 150,
        "exclude_colors": ["yellow"]
    }

    response = requests.post(f"{fastapi_tunnel.public_url}/recommend", json=payload)
    print(f"\nRecommendation status: {response.status_code}")
    print(f"Response: {response.text}")
except Exception as e:
    print(f"Error testing endpoints: {e}")

# Keep alive
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    ngrok.kill()
    backend_process.terminate()
    frontend_process.terminate()

Killing existing processes...
Starting FastAPI backend...
Starting ngrok tunnel for FastAPI...
FastAPI backend URL: https://7b5d7b081699.ngrok-free.app
Starting Streamlit frontend...
Starting ngrok tunnel for Streamlit...
Streamlit frontend URL: https://be9d9429c7fa.ngrok-free.app

Services are running...
Streamlit app: https://be9d9429c7fa.ngrok-free.app
FastAPI docs: https://7b5d7b081699.ngrok-free.app/docs
Backend health check: https://7b5d7b081699.ngrok-free.app/health

Health check status: 200
Response: {"status":"healthy"}

Recommendation status: 200
Response: {"outfit":[{"id":153,"name":"Crop Top #153","category":"top","color":"gray","color_hex":"#757575","style":"classic","reason":"Perfect for your hourglass body shape. Matches your classic style and the work event. Trendiness score: 0.84","brand":"Uniqlo","price":59,"description":"Perfect top for party","material":"Cotton","care_instructions":"Hand wash","compatibility_score":0.983836588371217},{"id":177,"name":"Tailored Trous